In [1]:
import os
import tweepy as tw
import pandas as pd
import emoji
import itertools
from textblob import TextBlob

In [2]:
T = pd.read_csv("Tweets.csv")

In [3]:
T.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
import re
def remove_punct(text):
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    text = emoji.demojize(text)
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub("(\w+:\/\/\S+)", " ", text)
    text = re.sub("[\.\,\!\?\:\;\-\=]", " ", text)
    return text

T['Clean Tweets'] = T['text'].apply(lambda x: remove_punct(x))

In [6]:
T.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,Clean Tweets
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),VirginAmerica What dhepburn said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),VirginAmerica plus youve added commercials to ...
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),VirginAmerica I didnt today Must mean I need t...
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),VirginAmerica its really aggressive to blast o...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),VirginAmerica and its a really big bad thing a...


In [7]:
TD = pd.DataFrame(data= T, columns =["airline_sentiment", "Clean Tweets"])
TD.head()

,airline_sentiment,Clean Tweets
0,neutral,VirginAmerica What dhepburn said
1,positive,VirginAmerica plus youve added commercials to ...
2,neutral,VirginAmerica I didnt today Must mean I need t...
3,negative,VirginAmerica its really aggressive to blast o...
4,negative,VirginAmerica and its a really big bad thing a...


In [8]:
twt = []
twt = TD["Clean Tweets"]

sent = []
polar = []


for x in twt:
    y = TextBlob(x)
    for z in y.sentences:
        w = z.sentiment.polarity
        if z.sentiment.polarity > 0:
            sent.append('1')
        elif z.sentiment.polarity < 0:
            sent.append('-1')
        else:
            sent.append('0')
    polar.append(w)

In [9]:
TD["score"] = sent

In [10]:
TD.head()

,airline_sentiment,Clean Tweets,score
0,neutral,VirginAmerica What dhepburn said,0
1,positive,VirginAmerica plus youve added commercials to ...,0
2,neutral,VirginAmerica I didnt today Must mean I need t...,-1
3,negative,VirginAmerica its really aggressive to blast o...,1
4,negative,VirginAmerica and its a really big bad thing a...,-1


In [11]:
twt = []
twt = TD["Clean Tweets"]
sent = []
polar = []

for x in twt:
    y = TextBlob(x)
    for z in y.sentences: 
        w = z.sentiment.polarity
        if z.sentiment.polarity > 0:
            sent.append('__label__POSITIVE')
        elif z.sentiment.polarity < 0:
            sent.append('__label__NEGATIVE')
        else:
            sent.append('__label__NEUTRAL')
    polar.append(w)

In [12]:
TD["L"]=sent
TD.head()

,airline_sentiment,Clean Tweets,score,L
0,neutral,VirginAmerica What dhepburn said,0,__label__NEUTRAL
1,positive,VirginAmerica plus youve added commercials to ...,0,__label__NEUTRAL
2,neutral,VirginAmerica I didnt today Must mean I need t...,-1,__label__NEGATIVE
3,negative,VirginAmerica its really aggressive to blast o...,1,__label__POSITIVE
4,negative,VirginAmerica and its a really big bad thing a...,-1,__label__NEGATIVE


In [15]:
TD["Tweet"]=TD["L"] + " " +TD["Clean Tweets"]
TD.head()

,airline_sentiment,Clean Tweets,score,L,Tweet
0,neutral,VirginAmerica What dhepburn said,0,__label__NEUTRAL,__label__NEUTRAL VirginAmerica What dhepburn said
1,positive,VirginAmerica plus youve added commercials to ...,0,__label__NEUTRAL,__label__NEUTRAL VirginAmerica plus youve adde...
2,neutral,VirginAmerica I didnt today Must mean I need t...,-1,__label__NEGATIVE,__label__NEGATIVE VirginAmerica I didnt today ...
3,negative,VirginAmerica its really aggressive to blast o...,1,__label__POSITIVE,__label__POSITIVE VirginAmerica its really agg...
4,negative,VirginAmerica and its a really big bad thing a...,-1,__label__NEGATIVE,__label__NEGATIVE VirginAmerica and its a real...


In [19]:
TDF = pd.DataFrame(data= TD, columns =["Tweet"])
TDF.head()

,Tweet
0,__label__NEUTRAL VirginAmerica What dhepburn said
1,__label__NEUTRAL VirginAmerica plus youve adde...
2,__label__NEGATIVE VirginAmerica I didnt today ...
3,__label__POSITIVE VirginAmerica its really agg...
4,__label__NEGATIVE VirginAmerica and its a real...


In [17]:
#TDF.to_csv(r'C:\Users\Paul\App-Data\Train_Data.csv')